# BioData Catalyst Powered by PIC-SURE: Identify stigmatizing variables

The purpose of this notebook is to identify stigmatizing variables in [BioData Catalyst Powered by PIC-SURE](https://picsure.biodatacatalyst.nhlbi.nih.gov/). Specifically, stigmatizing variables will be identified in PIC-SURE Authorized Access and removed for PIC-SURE Open Access.

For more information about stigmatizing variables, please view the [README.md](https://github.com/hms-dbmi/biodata_catalyst_stigmatizing_variables#biodata_catalyst_stigmatizing_variables).

---

### Prerequisites

This notebook assumes knowledge of the BioData Catalyst Powered by PIC-SURE platform, data structure, and API. For more information about the API, please visit the [Access to Data using PIC-SURE GitHub repository](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API).

Developer login credentials or access to all data in PIC-SURE Authorized Access is also required to ensure all variables are reviewed. 

### Connect to PIC-SURE

Be sure to save your user-specific token as `token.txt` prior to running the code.

In [ ]:
import pandas as pd
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git@new-search

import PicSureBdcAdapter

In [ ]:
# Uncomment production URL below for production environment
# PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
token_file = "token.txt"

with open(token_file, "r") as f:
    my_token = f.read()
    
bdc = PicSureBdcAdapter.Adapter(PICSURE_network_URL, my_token)


### Save all variables in PIC-SURE Authorized Access to DataFrame

In [ ]:
dictionary = bdc.useDictionary().dictionary() # Set up the dictionary
all_vars = dictionary.find()
all_variables = all_vars.dataframe() # Retrieve all variables you have access to

In [ ]:
#all_variables.head()

In [ ]:
#all_variables.columns

In [ ]:
clean_df = all_variables[["HPDS_PATH", "variable", "name", "description", 
                          "var_report_description", "var_name", "var_report_comment", "values",
                         "dataTableName", "dataTableDescription"]]
clean_df.head()

In [ ]:
def is_same(term1, term2):
    if term1 == term2:
        return term1
    else:
        final = str(term1)+" <<AND>> "+str(term2)
        return final

In [ ]:
final_var_info = []
final_dt_info = []
for i, path in enumerate(clean_df.HPDS_PATH): # Change to full df when ready 
    cur_var_info = []
    variable = clean_df.variable[i]
    if variable != '':
        cur_var_info.append(variable)
    name = is_same(clean_df.name[i], clean_df.var_name[i])
    if name != '':
        cur_var_info.append(name)
    description = is_same(clean_df.description[i], clean_df.var_report_description[i])
    if description != '':
        cur_var_info.append(description)
    comment = clean_df.var_report_comment[i]
    if comment != '':
        cur_var_info.append(comment)
    #print(cur_var_info)
    if len(cur_var_info) == 0:
        cur_var_info = "<<NO INFO AVAILABLE>>"
    final_var_info.append(cur_var_info)
    #clean_df.curated_var_info[i] = cur_var_info
    
    cur_dt_info = []
    dt_name = clean_df.dataTableName[i]
    if dt_name != '':
        cur_dt_info.append(dt_name)
    dt_desc = clean_df.dataTableDescription[i]
    if dt_desc != '':
        cur_dt_info.append(dt_desc)
    if len(cur_dt_info) == 0:
        cur_dt_info = "<<NO INFO AVAILABLE>>"
    final_dt_info.append(cur_dt_info)
#print(final_var_info)
clean_df['curated_var_info'] = final_var_info
clean_df['curated_dt_info'] = final_dt_info
df = clean_df[['HPDS_PATH', 'curated_var_info', 'curated_dt_info', 'values']]

In [ ]:
#df.head()

In [ ]:
stigmatizing_df = pd.read_csv("stigmatizing_terms/stigmatizing_keywords.tsv", sep="\t")
terms_included_df = pd.read_csv("stigmatizing_terms/inclusion_terms.tsv", sep='\t')
terms_excluded_df = pd.read_csv("stigmatizing_terms/revamped_exclusion.tsv", sep='\t')

In [ ]:
import re
def check_vars(varlist, df, exclude_vars=[]):
    stig_var_list = []
    #excluded_var_list = []
    for i in range(0, len(df["curated_var_info"])):
        mini = "N"
        for var in varlist:
            if mini == "N":
                if re.search(var, str(df['curated_var_info'][i]), re.IGNORECASE):
                #for ex in exclude_vars:
                #    if df['simplified_name'][i].lower() == ex:
                #        if df['simplified_name'][i] not in excluded_var_list:
                #            excluded_var_list.append(df['name'][i])
                #if df['name'][i] not in excluded_var_list:
                    #stig_var_list.append("Y")
                    mini = "Y"
            else:
                break
        stig_var_list.append(mini)
    df["need_review"] = stig_var_list
    return df[df.need_review == "Y"].reset_index(drop=True)
#test = check_vars(stigmatizing_df['Search keyword'], df)

In [ ]:
def automatic_inclusion(df, inclusion_terms):
    df["stigmatizing"] = "NA"
    for i in range(0, len(df.HPDS_PATH)):
        mini = "N"
        for var in inclusion_terms:
            if mini == "N":
                if re.search(var, str(df['curated_var_info'][i]), re.IGNORECASE):
                    mini = "Y"
                    df["stigmatizing"][i] = "Y"
            else:
                break
    return(df)

In [ ]:
#test2 = automatic_inclusion(test, terms_included_df["Terms to include"])
#test2.head()

In [ ]:
def exclude_terms(df, var_list):
    for i in range(0, len(df.HPDS_PATH)):
        if df.stigmatizing[i] == "NA":
            mini = "NA"
            for var in var_list:
                if mini == "NA":
                    if re.search(var, str(df['curated_var_info'][i]), re.IGNORECASE):
                        mini = "N"
                        df['stigmatizing'][i] = "N"
                else:
                    break
    return df
#test3 = exclude_terms(test2, terms_excluded_df["TERMS TO EXCLUDE"])

In [ ]:
from ast import literal_eval
def decide(df):
    stig_vars = []
    non_stig_vars = []
    for i in range(0, len(df.HPDS_PATH)):
        print(i)
        test = df.curated_var_info[i].replace(" nan]", " 'nan']")
        var_info = literal_eval(test)
        if df.stigmatizing[i] == "Y" and var_info[1].lower() not in stig_vars:
            newstring = ''.join([j for j in var_info[1].lower() if not j.isdigit()])
            stig_vars.append(newstring.lower())
            print("Adding", newstring, "to stig vars")
            #continue
        if df.stigmatizing[i] == "N" and var_info[1].lower() not in non_stig_vars:
            newstring = ''.join([j for j in var_info[1].lower() if not j.isdigit()])
            non_stig_vars.append(newstring.lower())
            print("Adding", newstring, "to non stig vars")
            #continue
        #if df.stigmatizing[i] == "NA":
        else:
            newstring = ''.join([j for j in var_info[1].lower() if not j.isdigit()])
            if newstring in stig_vars:
                result = "Y"
                print("Recording result ", i, "of", len(df.HPDS_PATH))
                df.stigmatizing[i] = result
                continue
            elif newstring in non_stig_vars:
                result = "N"
                print("Recording result", i, "of", len(df.HPDS_PATH))
                df.stigmatizing[i] = result
                continue
            else:
                print("Variable", i, "of", len(df.HPDS_PATH))
                print(var_info)
                result = input("Stigmatizing? Y/N/more: ")
                if result == "more":
                    print(df.curated_dt_info[i])
                    result = input("Table info. Stigmatizing? Y/N: ")
                if result == "pause":
                    print("Pausing stigmatizing variable identification")
                    return(df)
                if result == "Y":
                    #newstring = ''.join([j for j in var_info[1].lower() if not j.isdigit()])
                    stig_vars.append(newstring.lower())
                if result == "N":
                    #newstring = ''.join([j for j in var_info[1].lower() if not j.isdigit()])
                    non_stig_vars.append(newstring.lower())
                df.stigmatizing[i] = result
            #return(stig_vars)
        #else:
        #    continue
    print("Stigmatizing variables complete.")
    return(df)

In [ ]:
def checkpoint(df, include, exclude):
    for i in range(0, len(df.HPDS_PATH)):
        if df.stigmatizing[i] == "NA":
            mini = "NA"
            for var in include:
                if mini == "NA":
                    if re.search(var.lower(), str(df['curated_var_info'][i]), re.IGNORECASE):
                        mini = "N"
                        df['stigmatizing'][i] = "Y"
                else:
                    break
            if mini == "NA":
                for var in exclude:
                    if mini == "NA":
                        if re.search(var.lower(), str(df["curated_var_info"][i]), re.IGNORECASE):
                            mini = "N"
                            df['stigmatizing'][i] = "N"
                    else:
                        break
    return df

In [ ]:
final_output = 'stigmatizing_variable_results/REVAMP_stigmatizing_variables_decisions.txt'
df_inc_exc = pd.read_csv(final_output, sep='\t')
df_inc_exc.fillna('NA', inplace=True)

In [ ]:
#df_inc_exc = checkpoint(df_inc_exc, terms_included_df["Terms to include"], terms_excluded_df["TERMS TO EXCLUDE"])
#df_inc_exc = checkpoint(df_inc_exc, ["LOW EDUCATION", "ILLITERACY", 'bcp', 'hrt', 'hormone replacement therapy'], 
#                        ['macular', 'cholesterol', 'hypopnea', 'mitral', 'gallbladder', 'cheese', 'z-score', 
#                         'x-ray', 'blood processing', 'migraine', 'antigout','antiulcer'])
df_inc_exc = checkpoint(df_inc_exc, ['birth control pill', 'Anti-depressants', 'Antidepressants', 'liquor'], 
                        ["orthopnea", 'carotid', 'extremit', 'limb',
                                        'ventricular', 'Hachinski', 'cortex', 'cuneus',
                                        'Reader-trend-adjusted ultrasound',
                                        'Ultrasound Imputed Values', 'heart failure',
                                        'chlamydia pneumoniae', 'Citrated plasma', 'both lungs', 'st segment', 
                                        'electrophoresis', 'pressed for time', 'form version', 'gall bladder',
                                        'glucose', 'glaucoma', 'Indicator for presence of form',
                                        'Intrisicoid'])

In [ ]:
sum(df_inc_exc.stigmatizing == "NA") #26233

In [ ]:
df_final = decide(df_inc_exc)

In [ ]:
final_output = 'stigmatizing_variable_results/REVAMP_stigmatizing_variables_decisions.txt'
df_final.to_csv(final_output, sep='\t', header=True, index=False)

## Export stig vars

In [1]:
import pandas as pd
final_output = 'stigmatizing_variable_results/REVAMP_stigmatizing_variables_decisions_reviewed.xlsx - REVAMP_stigmatizing_variables_d.tsv'
df = pd.read_csv(final_output, sep='\t')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df.head()

,HPDS_PATH,curated_var_info,curated_dt_info,values,stigmatizing,to review
0,\phs000200\pht003400\phv00192428\ICDCODE\,"['ICDCODE', 'ICD-O-2 site code', ""All centrall...",['outc_bc_cad_rel4'],"{'07.9': 'Parotid gland', '71.1': 'Frontal lob...",N,NaN
1,\phs000200\pht003401\phv00192454\ICDCODE\,"['ICDCODE', 'ICD-O-2 site code', ""All centrall...",['outc_bc_rel4'],"{'07.9': 'Parotid gland', '71.1': 'Frontal lob...",N,NaN
2,\phs000200\pht003403\phv00192894\ICDCODE\,"['ICDCODE', 'ICD-O-2 site code', ""All centrall...",['outc_cancer_cad_rel4'],"{'07.9': 'Parotid gland', '71.1': 'Frontal lob...",N,NaN
3,\phs000007\pht002350\phv00162315\OTHER1\,"['OTHER1', 'FFQ: OTHER FOOD 1', 'Use spreadshe...","['vr_ffreq_ex08_1_0615s', ""Food Frequency Ques...","{'88': 'High Protein Diet Supplement, HP Hot C...",N,NaN
4,\phs000007\pht000680\phv00070037\OTHER1\,"['OTHER1', 'FFQ: OTHER FOOD 1', nan]","['ffreq1_5s', 'Food Frequency Questionnaire Da...","{'88': 'High Protein Diet Supplement, HP Hot C...",N,NaN


In [9]:
# FIlter to stig vars
stigvars = df[df.stigmatizing == "Y"].HPDS_PATH.reset_index(drop=True)

In [10]:
stigvars.head()

0    \phs000007\pht003094\phv00177292\g3b0073\
1       \phs000007\pht005140\phv00254551\j901\
2       \phs000007\pht005140\phv00254546\j895\
3    \phs000007\pht003094\phv00177640\g3b0650\
4    \phs000007\pht003094\phv00177646\g3b0656\
Name: HPDS_PATH, dtype: object

In [ ]:
out = "stigmatizing_variable_results/REVAMP_stigmatizing_variables.txt"
df_final.to_csv(out, sep='\t', header=False, index=False)